In [1]:
import torch
import sys

sys.path.append("../pylon/")
from lazy_tensor import ConstLazyTensor, ConstShapedLazyTensor, LazyTensor 

In [2]:
def rand_tensor(dims, isBool=True):
    """ Create a torch.Tensor instance
    
        dims: dimension
        isBool: if it's a boolean tensor
    """
    if isBool:
        return torch.rand(dims) < .5
    return torch.rand(dims)

In [3]:
def print_all(lt, lt_parsed_tensor, rt):
    assert torch.equal(lt_parsed_tensor, rt)
    print("lt\n", lt, "\nParsed tensor of lt\n", lt_parsed_tensor, "\nrt\n", rt, "\n")

In [4]:
def parse_and_print(pn):
    lt, rt = parse_pn(pn)
    print_all(lt, lt.tensor(), rt)

### Create a ConstLazyTensor

In [5]:
t = rand_tensor((2,3), True)
print(t)
clt = ConstLazyTensor(t)
print(clt)

tensor([[False,  True, False],
        [False,  True,  True]])
ConstLazyTensor: value=
tensor([[False,  True, False],
        [False,  True,  True]])


In [6]:
t = rand_tensor((2,3), False)
print(t)
clt = ConstLazyTensor(t)
print(clt)

tensor([[0.2268, 0.9003, 0.4085],
        [0.4528, 0.6251, 0.9194]])
ConstLazyTensor: value=
tensor([[0.2268, 0.9003, 0.4085],
        [0.4528, 0.6251, 0.9194]])


All operators on ConstLazyTensor or ConstShapedLazyTensor creates a LazyTensor tree with correspending leaf nodes

### Unary operator: not on ConstLazyTensor

In [7]:
clt = ConstLazyTensor(rand_tensor((2,3), True))
print(clt)
lt = clt.logical_not()
print(lt)

ConstLazyTensor: value=
tensor([[False, False, False],
        [ True, False,  True]])
LazyTensor:logical_not
  ConstLazyTensor: value=
tensor([[False, False, False],
        [ True, False,  True]])



### Binary operator on ConstLazyTensor_s creates a LazyTensor 

In [8]:
clt_0 = ConstLazyTensor(rand_tensor((2,2), True))
clt_1 = ConstLazyTensor(rand_tensor((2,2), True))
lt = clt_0.logical_and(clt_1)
print(lt)

LazyTensor:logical_and
  ConstLazyTensor: value=
tensor([[False,  True],
        [ True,  True]])
  ConstLazyTensor: value=
tensor([[ True, False],
        [False,  True]])



In [9]:
lt = clt_0.logical_or(clt_1)
print(lt)

LazyTensor:logical_or
  ConstLazyTensor: value=
tensor([[False,  True],
        [ True,  True]])
  ConstLazyTensor: value=
tensor([[ True, False],
        [False,  True]])



In [10]:
# softmax on dim = -1
lt = ConstLazyTensor(torch.randn(2, 3)).softmax(-1)
print(lt)

LazyTensor:softmax
  ConstLazyTensor: value=
tensor([[-0.8621,  0.5877,  0.3988],
        [-0.0379,  0.8082,  1.7811]])
  -1



## 2-level nested operators 

In [11]:
clt_0 = ConstLazyTensor(rand_tensor((2,2), True))
clt_1 = ConstLazyTensor(rand_tensor((2,2), True))
lt_0 = clt_0.logical_and(clt_1)
lt_1 = clt_0.logical_or(clt_1)
nested_lt = lt_0.logical_and(lt_1)
print(nested_lt)

LazyTensor:logical_and
  LazyTensor:logical_and
    ConstLazyTensor: value=
tensor([[ True,  True],
        [False,  True]])
    ConstLazyTensor: value=
tensor([[True, True],
        [True, True]])

  LazyTensor:logical_or
    ConstLazyTensor: value=
tensor([[ True,  True],
        [False,  True]])
    ConstLazyTensor: value=
tensor([[True, True],
        [True, True]])




### 3-level nested

In [12]:
clt_0 = ConstLazyTensor(rand_tensor((2,2), True))
clt_1 = ConstLazyTensor(rand_tensor((2,2), True))
lt_0 = clt_0.logical_and(clt_1)
lt_1 = clt_0.logical_or(clt_1)
nested_l2_lt = lt_0.logical_and(lt_1)
nested_lt = nested_l2_lt.logical_not()
print(nested_lt)

LazyTensor:logical_not
  LazyTensor:logical_and
    LazyTensor:logical_and
      ConstLazyTensor: value=
tensor([[False,  True],
        [ True, False]])
      ConstLazyTensor: value=
tensor([[False,  True],
        [False,  True]])

    LazyTensor:logical_or
      ConstLazyTensor: value=
tensor([[False,  True],
        [ True, False]])
      ConstLazyTensor: value=
tensor([[False,  True],
        [False,  True]])





### __getitem__  using [ ]

In [13]:
lt = ConstLazyTensor(rand_tensor((2), False))
lt[1].tensor()

tensor(0.4089)

In [14]:
lt = ConstLazyTensor(rand_tensor((3,5,4), False))

In [15]:
# single index
print(lt[1], "\n\n", lt[1].tensor())

LazyTensor:__getitem__
  ConstLazyTensor: value=
tensor([[[0.1629, 0.6182, 0.3786, 0.9823],
         [0.5548, 0.4734, 0.2662, 0.8199],
         [0.8828, 0.2802, 0.3103, 0.1418],
         [0.3761, 0.6347, 0.2251, 0.0356],
         [0.9169, 0.9758, 0.3606, 0.7458]],

        [[0.2239, 0.3474, 0.2259, 0.9240],
         [0.5251, 0.8135, 0.8261, 0.5066],
         [0.1127, 0.4026, 0.2899, 0.6820],
         [0.3997, 0.9630, 0.5086, 0.9928],
         [0.1767, 0.9564, 0.8949, 0.5115]],

        [[0.9330, 0.0483, 0.9545, 0.0320],
         [0.0517, 0.3617, 0.5920, 0.1433],
         [0.3785, 0.8532, 0.7394, 0.7619],
         [0.1519, 0.1175, 0.1724, 0.2888],
         [0.3189, 0.3028, 0.7001, 0.5981]]])
  1
 

 tensor([[0.2239, 0.3474, 0.2259, 0.9240],
        [0.5251, 0.8135, 0.8261, 0.5066],
        [0.1127, 0.4026, 0.2899, 0.6820],
        [0.3997, 0.9630, 0.5086, 0.9928],
        [0.1767, 0.9564, 0.8949, 0.5115]])


In [16]:
# tuple of 2 indices
print(lt[1,4], "\n\n", lt[1,4].tensor())

LazyTensor:__getitem__
  ConstLazyTensor: value=
tensor([[[0.1629, 0.6182, 0.3786, 0.9823],
         [0.5548, 0.4734, 0.2662, 0.8199],
         [0.8828, 0.2802, 0.3103, 0.1418],
         [0.3761, 0.6347, 0.2251, 0.0356],
         [0.9169, 0.9758, 0.3606, 0.7458]],

        [[0.2239, 0.3474, 0.2259, 0.9240],
         [0.5251, 0.8135, 0.8261, 0.5066],
         [0.1127, 0.4026, 0.2899, 0.6820],
         [0.3997, 0.9630, 0.5086, 0.9928],
         [0.1767, 0.9564, 0.8949, 0.5115]],

        [[0.9330, 0.0483, 0.9545, 0.0320],
         [0.0517, 0.3617, 0.5920, 0.1433],
         [0.3785, 0.8532, 0.7394, 0.7619],
         [0.1519, 0.1175, 0.1724, 0.2888],
         [0.3189, 0.3028, 0.7001, 0.5981]]])
  (1, 4)
 

 tensor([0.1767, 0.9564, 0.8949, 0.5115])


In [17]:
# tuple of 3 indices
print(lt[2,3,2], "\n\n",lt[2,3,2].tensor())

LazyTensor:__getitem__
  ConstLazyTensor: value=
tensor([[[0.1629, 0.6182, 0.3786, 0.9823],
         [0.5548, 0.4734, 0.2662, 0.8199],
         [0.8828, 0.2802, 0.3103, 0.1418],
         [0.3761, 0.6347, 0.2251, 0.0356],
         [0.9169, 0.9758, 0.3606, 0.7458]],

        [[0.2239, 0.3474, 0.2259, 0.9240],
         [0.5251, 0.8135, 0.8261, 0.5066],
         [0.1127, 0.4026, 0.2899, 0.6820],
         [0.3997, 0.9630, 0.5086, 0.9928],
         [0.1767, 0.9564, 0.8949, 0.5115]],

        [[0.9330, 0.0483, 0.9545, 0.0320],
         [0.0517, 0.3617, 0.5920, 0.1433],
         [0.3785, 0.8532, 0.7394, 0.7619],
         [0.1519, 0.1175, 0.1724, 0.2888],
         [0.3189, 0.3028, 0.7001, 0.5981]]])
  (2, 3, 2)
 

 tensor(0.1724)


In [18]:
# 3 dots or Ellipsis
print(lt[...], "\n\n", lt[...].tensor())

LazyTensor:__getitem__
  ConstLazyTensor: value=
tensor([[[0.1629, 0.6182, 0.3786, 0.9823],
         [0.5548, 0.4734, 0.2662, 0.8199],
         [0.8828, 0.2802, 0.3103, 0.1418],
         [0.3761, 0.6347, 0.2251, 0.0356],
         [0.9169, 0.9758, 0.3606, 0.7458]],

        [[0.2239, 0.3474, 0.2259, 0.9240],
         [0.5251, 0.8135, 0.8261, 0.5066],
         [0.1127, 0.4026, 0.2899, 0.6820],
         [0.3997, 0.9630, 0.5086, 0.9928],
         [0.1767, 0.9564, 0.8949, 0.5115]],

        [[0.9330, 0.0483, 0.9545, 0.0320],
         [0.0517, 0.3617, 0.5920, 0.1433],
         [0.3785, 0.8532, 0.7394, 0.7619],
         [0.1519, 0.1175, 0.1724, 0.2888],
         [0.3189, 0.3028, 0.7001, 0.5981]]])
  ...
 

 tensor([[[0.1629, 0.6182, 0.3786, 0.9823],
         [0.5548, 0.4734, 0.2662, 0.8199],
         [0.8828, 0.2802, 0.3103, 0.1418],
         [0.3761, 0.6347, 0.2251, 0.0356],
         [0.9169, 0.9758, 0.3606, 0.7458]],

        [[0.2239, 0.3474, 0.2259, 0.9240],
         [0.5251, 0.8135, 0.82

### Boolean mask

In [19]:
y = rand_tensor((2,2), False)
x = rand_tensor((2,2), False)
print(y)
print(x)
x==1

tensor([[0.1699, 0.8443],
        [0.1854, 0.5940]])
tensor([[0.8036, 0.7753],
        [0.9398, 0.2031]])


tensor([[False, False],
        [False, False]])

In [22]:
y_lt = ConstLazyTensor(y)
x_lt = ConstLazyTensor(x)
# y[x==1]
print(y_lt[x_lt==1])
print()
print(y_lt[x_lt==1] == 0)

LazyTensor:__getitem__
  ConstLazyTensor: value=
tensor([[0.1699, 0.8443],
        [0.1854, 0.5940]])
  LazyTensor:eq
    ConstLazyTensor: value=
tensor([[0.8036, 0.7753],
        [0.9398, 0.2031]])
    1



LazyTensor:eq
  LazyTensor:__getitem__
    ConstLazyTensor: value=
tensor([[0.1699, 0.8443],
        [0.1854, 0.5940]])
    LazyTensor:eq
      ConstLazyTensor: value=
tensor([[0.8036, 0.7753],
        [0.9398, 0.2031]])
      1


  0

